<h1 style="color:#be4f0c; font-family:Times New Roman">VIX & SPX JOINT CALIBRATION - SKEWED 2 FACTOR BERGOMI </h1>
<h4 style="color:#393939; font-style: italic; font-family:Times New Roman">authors: @lg2894 Louis Guigo - @mak809 Michael Ang</h4>
<h3 style="color:#001f3f; font-family:Times New Roman">Imports</h3>

In [1]:
import time
import gc
import numba

import math
import numpy as np
import pandas as pd

from scipy import optimize, stats, interpolate
from scipy.integrate import quad

from bqplot import pyplot as qplt
from IPython.display import display

#TODO: pip install git+git://github.com/khrapovs/impvol
from impvol import imp_vol, lfmoneyness, impvol_bisection

<h3 style="color:#001f3f; font-family:Times New Roman">Dark theme</h3>

In [2]:
%%javascript
document.body.classList.add('theme-dark')

<IPython.core.display.Javascript object>

<h1 style="color:#ec7f37; font-family:Times New Roman">PART 1: Skewed model simulation and calibration to VIX and SPX options & futures</h1>
<h3 style="color:#001f3f; font-family:Times New Roman">Model simulation functions</h3>

In [3]:
# skewed model parameters arrays used in the simulation (gamma, beta, zeta)
def create_array_time_dependent_parameters(times, vix_maturities_array, gamma, beta, zeta):
    """
    Create arrays of piecewise constant parameters of the skewed 2 factor Bergomi mode (gamma, beta, zeta)
    """
    
    # allocations
    tau = 30.0/365.0 # vix time interval
    gamma_t = np.empty_like(times)
    beta_t = np.empty_like(times)
    zeta_t = np.empty_like(times)
    
    # before first VIX future maturity
    gamma_t[times < vix_maturities_array[0]] = gamma[0]
    beta_t[times < vix_maturities_array[0]] = beta[0]
    zeta_t[times < vix_maturities_array[0]] = zeta[0]
    # from first to second
    gamma_t[(times >= vix_maturities_array[0]) &\
            (times < vix_maturities_array[0]+tau)] = gamma[0]
    beta_t[(times >= vix_maturities_array[0]) &\
           (times < vix_maturities_array[0]+tau)] = beta[0]
    zeta_t[(times >= vix_maturities_array[0]) &\
           (times < vix_maturities_array[0]+tau)] = zeta[0]
    
    # intermediate
    for j in range(1,vix_maturities_array.size-1):
        gamma_t[(times >= vix_maturities_array[j]) &\
                (times >= vix_maturities_array[j-1]+tau) &\
                (times < vix_maturities_array[j]+tau)] = gamma[j]
        beta_t[(times >= vix_maturities_array[j]) &\
               (times >= vix_maturities_array[j-1]+tau) &\
               (times < vix_maturities_array[j]+tau)] = beta[j]
        zeta_t[(times >= vix_maturities_array[j]) &\
               (times >= vix_maturities_array[j-1]+tau) &\
               (times < vix_maturities_array[j]+tau)] = zeta[j]
        # if 5 days empty, half sums
        gamma_t[(times < vix_maturities_array[j]) &\
                (times >= vix_maturities_array[j-1]+tau)] = 0.5 * (gamma[j-1] + gamma[j])
        beta_t[(times < vix_maturities_array[j]) &\
               (times >= vix_maturities_array[j-1]+tau)] = 0.5 * (beta[j-1] + beta[j])
        zeta_t[(times < vix_maturities_array[j]) &\
               (times >= vix_maturities_array[j-1]+tau)] = 0.5 * (zeta[j-1] + zeta[j])
    
    # last maturity
    j = vix_maturities_array.size - 1
    gamma_t[(times >= vix_maturities_array[j]) &\
            (times >= vix_maturities_array[j-1]+tau)] = gamma[j]
    beta_t[(times >= vix_maturities_array[j]) &\
           (times >= vix_maturities_array[j-1]+tau)] = beta[j]
    zeta_t[(times >= vix_maturities_array[j]) &\
           (times >= vix_maturities_array[j-1]+tau)] = zeta[j]
    # if 5 days empty, half sums
    gamma_t[(times < vix_maturities_array[j]) &\
            (times >= vix_maturities_array[j-1]+tau)] = 0.5 * (gamma[j-1] + gamma[j])
    beta_t[(times < vix_maturities_array[j]) &\
           (times >= vix_maturities_array[j-1]+tau)] = 0.5 * (beta[j-1] + beta[j])
    zeta_t[(times < vix_maturities_array[j]) &\
           (times >= vix_maturities_array[j-1]+tau)] = 0.5 * (zeta[j-1] + zeta[j])
    
    return (gamma_t, beta_t, zeta_t)

In [4]:
# xi_t^T in the skewed 2 factor model - [T,x1,x2 = vectors]
def instantaneous_variance_x_t_T(t, x1, x2, T, theta, alphatheta, k1, k2, rho12, nu,\
                     xi0, gamma_t, beta_t, zeta_t, omega_t, times):

    # t of xi_t^T
    maturity_time_index = np.abs(times - t).argmin()
    # T of xi_t^T
    T_time_index = (np.abs(times[:, np.newaxis] - T[np.newaxis, :])).argmin(axis=0)
    
    # value of time dependent parameters at T
    omega, gamma, beta, zeta = omega_t[T_time_index].reshape((-1,1)),\
                               gamma_t[T_time_index].reshape((-1,1)),\
                               beta_t[T_time_index].reshape((-1,1)),\
                               zeta_t[T_time_index].reshape((-1,1))
    
    chi = (alphatheta**2) * (((1-theta)**2) * np.exp(-2 * k1 * (T-t)) *\
                                 (1 - np.exp(-2 * k1 * t)) / (2 * k1)\
                             + (theta**2) * np.exp(-2 * k2 * (T-t)) *\
                                 (1 - np.exp(-2 * k2 * t)) / (2 * k2) +\
                             2*theta*(1-theta)*rho12 * np.exp(-(k1+k2) * (T-t)) *\
                                     (1 - np.exp(-(k1+k2)*t)) / (k1+k2)\
                             )
    
    x = alphatheta * ((1-theta) * np.exp(-k1 * (T[:,np.newaxis] - t)) * x1[np.newaxis,:] +\
                      theta * np.exp(-k2 * (T[:,np.newaxis] - t)) * x2[np.newaxis,:])

    return(xi0[T_time_index].reshape((-1,1)) * \
           ((1-gamma)*np.exp(omega*x -0.5*(omega**2)*chi.reshape((-1,1))) + \
            gamma*np.exp(beta*omega*x - 0.5*((omega*beta)**2)*chi.reshape((-1,1)))))


# VIX - basket of inst. var. (int (xi_T^u du) u=T to T+tau) - Riemman sums
def vix_in_model(x1, x2, T, theta, alphatheta, k1, k2, rho12, nu, xi0,\
                 gamma_t, beta_t, zeta_t, omega_t, times,\
                 nb_points_riemman_sum=40):
    
    # vix time interval
    tau = 30.0/365.0
    
    # riemman nodes
    times_inside = np.linspace(T, T+tau, nb_points_riemman_sum)
    value_at_nodes = instantaneous_variance_x_t_T(T, x1, x2, times_inside,\
                                            theta, alphatheta, k1, k2, rho12, nu, xi0,\
                                            gamma_t, beta_t, zeta_t, omega_t, times)
    # sum trapezes
    return(100 * np.sqrt(np.sum((value_at_nodes[1:,:] + value_at_nodes[:-1,:]) / \
                                (2*(value_at_nodes.shape[0]-1)), axis=0)))

In [5]:
# simulate increments of model processes (X1_t, X2_t, S_t)
def simulate_processes_increments(times, nb_paths, nb_steps,\
                             k1, k2, nu, rhoS1, rhoS2, rho12):
    
    # draw std normal gaussian vector realizations
    gaussian_3d_N01 = np.random.normal(size=(nb_paths,nb_steps,3))
    
    def covariance_matrix_function(dt):

        I1 = (1 - np.exp(-k1*dt))/(k1*dt)
        I2 = (1 - np.exp(-k2*dt))/(k2*dt)
        
        I1I1 = (1 - np.exp(-2*k1*dt))/(2*k1*dt)
        I2I2 = (1 - np.exp(-2*k2*dt))/(2*k2*dt)
        I1I2 = rho12*(1 - np.exp(-(k1+k2)*dt))/((k1+k2)*dt)
        

        cov_matrix = dt * np.matrix([[I1I1, I1I2, rhoS1*I1],\
                                     [I1I2, I2I2, rhoS2*I2],\
                                     [rhoS1*I1, rhoS2*I2, 1.0]])

        return cov_matrix
    
    increments = np.empty((nb_paths,nb_steps,3))
    
    for j in range(1, nb_steps):
        dt = times[j] - times[j-1]
        
        # instantaneous covariance matrix
        cov_t = covariance_matrix_function(dt)
        
        # singular value decomposition
        U, s, V = np.linalg.svd(cov_t)
        svd = np.dot(np.dot(U, np.diag(np.power(s, 0.5))), V)
        increments[:,j] = np.einsum('kj,ij->ki', gaussian_3d_N01[:,j], svd)
        
        # factors increments (o-u processes) 
        increments[:,j,0] += np.exp(-k1*dt) * increments[:,j-1,0]
        increments[:,j,1] += np.exp(-k2*dt) * increments[:,j-1,1]
        
    return(increments)

# Instantaneous variances + stock simultaneously
def xi_stock_and_vix(increments, times, nb_paths,\
                          vix_maturity, vix_maturities,\
                          k1, k2, nu, rho12, theta, xi0, gamma, beta, zeta, S0, r, q,\
                          return_vix_path=True, return_stock_path=True):
    
    # constants (vix time interval, alpha_theta)
    tau = 30.0 / 365.0
    a = 1 / np.sqrt((1-theta)**2 + theta**2 + 2*rho12*theta*(1-theta))
    
    # time dependent parameters
    nb_steps = len(times)
    gamma_t, beta_t, zeta_t = create_array_time_dependent_parameters(times,\
                                                    vix_maturities, gamma, beta, zeta)
    nu_t = np.array([nu * zeta_t[i] / ((1-gamma_t[i]) + beta_t[i] * gamma_t[i])\
                     for i in range(nb_steps)])
    omega_t = 2 * nu_t
    
    # allocations
    xi_paths = np.empty((nb_paths,nb_steps))
    stock_paths = np.empty((nb_paths,nb_steps))
    vix_paths = np.empty((1,nb_paths))
    stock_paths[:,0] = S0
    
    # stock paths computation
    if return_stock_path:
        # t = 0
        k = np.array([[k1], [k2]])
        norm = a*np.array([[1-theta], [theta]])
        chi_t_vector = a**2 * ((1-theta)**2 * (1 - np.exp(-2*k1*times))/(2*k1)
                            + theta**2 * (1 - np.exp(-2*k2*times))/(2*k2)
                            + 2*rho12*theta*(1-theta) * (1-np.exp(-(k1+k2)*times))/(k1+k2)
                            )
        little_xt = (norm*[increments[:,0,0], increments[:,0,1]]).sum(axis=0)
        xi_paths[:,0] = xi0[0] * ((1-gamma_t[0])*np.exp(omega_t[0]*little_xt\
                                                      - 0.5*(omega_t[0]**2)*chi_t_vector[0]) \
                                        + gamma_t[0]*np.exp(beta_t[0]*omega_t[0]*little_xt\
                                                - 0.5*((omega_t[0]*beta_t[0])**2)*chi_t_vector[0]))
        
        for j in range(1,nb_steps):
            dt = times[j] - times[j-1]

            # instantaneous variance
            little_xt = (norm*[increments[:,j,0], increments[:,j,1]]).sum(axis=0)
            xi_paths[:,j] = xi0[j] * ((1-gamma_t[j])*np.exp(omega_t[j]*little_xt\
                                                    - 0.5*(omega_t[j]**2)*chi_t_vector[j]) + \
                                       gamma_t[j]*np.exp(beta_t[j]*omega_t[j]*little_xt\
                                                - 0.5*((omega_t[j]*beta_t[j])**2)*chi_t_vector[j]))

            # S_t via log euler scheme
            stock_paths[:,j] = stock_paths[:,j-1] * np.exp(np.sqrt(xi_paths[:,j-1]) * increments[:,j,2]\
                                                        + (r(times[j])-q(times[j]) - 0.5*xi_paths[:,j-1])*dt)

    # vix computation
    vix_maturity_index = np.abs(times - vix_maturity).argmin()
    vix_maturity_times = times[vix_maturity_index]
    if return_vix_path:
        vix_paths = vix_in_model(increments[:,vix_maturity_index,0], increments[:,vix_maturity_index,1],\
                                    vix_maturity, theta, a, k1, k2, rho12, nu, xi0,\
                                    gamma_t, beta_t, zeta_t, omega_t, times)

    return(xi_paths, stock_paths, vix_paths)

<h3 style="color:#001f3f; font-family:Times New Roman">Test simulation functions</h3>

In [6]:
# grid and montecarlo parameters
nsteps_per_year = 200
maturity = 1.0
nb_paths = 100000
nb_steps = int(maturity * nsteps_per_year)
times = np.linspace(0, maturity, nb_steps)

# model parameters
k1 = 5.35
k2 = 0.28
nu = 1.74
rhoS1 = 0.0
rhoS2 = 0.0
chi = 0.0
rho12 = rhoS1*rhoS2 + chi*np.sqrt(1-rhoS1**2)*np.sqrt(1-rhoS2**2)
theta = 0.245

# simulate increments
increments = simulate_processes_increments(times, nb_paths, nb_steps,\
                                           k1, k2, nu,\
                                           rhoS1, rhoS2, rho12)

In [7]:
# market as of october 8
vix_maturities = np.array([8, 36, 64, 92, 120, 148]) / 365
vix_maturity = vix_maturities[1]
# to be found (taken deterministic for now)
xi0 = lambda t: 0.2**2 * np.ones_like(t)
S0 = 100.0
r = lambda t: 0.0
q = lambda t: 0.0

In [8]:
# time dependent parameters (julien's input here)
gamma_beta_zeta = np.zeros((len(vix_maturities),3))
gamma_beta_zeta[:,0] = np.array([0.6]*len(vix_maturities))
gamma_beta_zeta[:,1] = np.array([0.2]*len(vix_maturities))
gamma_beta_zeta[:,2] = np.array([0.5]*len(vix_maturities))

In [9]:
# instantaneous variances and spot paths + vix at maturity vix_maturity 
xi, stock, vix = xi_stock_and_vix(increments, times, nb_paths,\
                          vix_maturity, vix_maturities,\
                          k1, k2, nu, rho12, theta,\
                          xi0(times), gamma_beta_zeta[:,0], gamma_beta_zeta[:,1], gamma_beta_zeta[:,2],\
                          S0, r, q)

<h3 style="color:#001f3f; font-family:Times New Roman">VIX and SPX simulated smiles</h3>

In [10]:
# European options price from paths (montecarlo)
# spx
def european_call_spx(price_paths, times, maturity, strikes, r):
    maturity_index = np.abs(times - maturity).argmin()
    maturity_price = price_paths[:, maturity_index]
    payoffs = np.matrix([np.where((maturity_price > k), maturity_price - k, 0.0) for k in strikes])
    price = np.exp(-r(maturity)) * payoffs.mean(1)
    return(price)
# vix
def european_call_vix(price_series, strikes):
    payoffs = np.matrix([np.where((price_series > k), price_series - k, 0.0) for k in strikes])
    price = payoffs.mean(1)
    return(price)

# range of strikes studied
# spx
min_strike_spx, max_strike_spx = 0.60, 1.40
range_strike_spx = [float(i) for i in range(int(S0*min_strike_spx), int(S0*max_strike_spx))]
# vix
min_strike_vix, max_strike_vix = 0.75, 2.50
future_vix = european_call_vix(vix, [0])[0,0]
range_strike_vix = [float(i) for i in range(int(future_vix*min_strike_vix), int(future_vix*max_strike_vix))]

In [11]:
# SPX call prices
price_spx_options = np.ravel(european_call_spx(stock, times, maturity, range_strike_spx, r))

price_spx_options_fig = qplt.figure(title='SPX call prices - skewed 2 factor Bergomi', 
           scales={"x": qplt.LinearScale()},
           min_width=800, min_height=300, width=1000, height=400,
           padding_x=.02, padding_y=0.01,
           align='right',
           legend_location='right')
price_spx_options_fig.layout.width = '700px'

qplt.plot(np.array(range_strike_spx), price_spx_options,\
          colors=['#FF851B'], marker='circle', marker_size=16)
qplt.axes()['x'].label = 'Strike'
qplt.axes()['y'].label = 'Price'

qplt.show()

# SPX call implied volatilities
moneyness = lfmoneyness(S0, range_strike_spx, r(maturity), maturity)
iv_spx_options = impvol_bisection(moneyness, maturity, price_spx_options/S0, True)

iv_spx_options_fig = qplt.figure(title='SPX call implied volatilities - skewed 2 factor Bergomi', 
           scales={"x": qplt.LinearScale()},
           min_width=800, min_height=300, width=1000, height=400,
           padding_x=.02, padding_y=0.01,
           align='right',
           legend_location='right')
iv_spx_options_fig.layout.width = '700px'

qplt.plot(range_strike_spx, iv_spx_options, colors=['#0074D9'], marker='cross', marker_size=16)
qplt.axes()['x'].label = 'Strike'
qplt.axes()['y'].label = 'Implied vol'

qplt.show()

A Jupyter Widget

A Jupyter Widget

In [12]:
# VIX call prices
price_vix_options = np.ravel(european_call_vix(vix, range_strike_vix))

price_vix_options_fig = qplt.figure(title='VIX call prices - skewed 2 factor Bergomi',
           scales={"x": qplt.LinearScale()},
           min_width=800, min_height=300, width=1000, height=400,
           padding_x=.02, padding_y=0.01,
           align='right',
           legend_location='right')
price_vix_options_fig.layout.width = '700px'

qplt.plot(np.ravel(range_strike_vix)/100, price_vix_options, colors=['#FFDC00'],\
          marker='circle', marker_size=16)
qplt.axes()['x'].label = 'Strike'
qplt.axes()['y'].label = 'Price'

qplt.show()

# VIX call implied volatilities
vix_moneyness = lfmoneyness(future_vix, range_strike_vix, 0.0, vix_maturity)
iv_vix_options = impvol_bisection(vix_moneyness, vix_maturity, price_vix_options/future_vix, True)

iv_vix_options_fig = qplt.figure(title='VIX call implied volatilities - skewed 2 factor Bergomi',
           scales={"x": qplt.LinearScale()},
           min_width=800, min_height=300, width=1000, height=400,
           padding_x=.02, padding_y=0.01,
           align='right',
           legend_location='right')
iv_vix_options_fig.layout.width = '700px'

qplt.plot(np.ravel(range_strike_vix)/100, iv_vix_options, colors=['#7FDBFF'],\
          marker='circle', marker_size=16)
qplt.vline(level=future_vix/100, colors=['#7FDBFF'], linestyle=['o-'],\
           labels=['Calibrated VIX future '+str(round(future_vix,2))], display_legend=True)
qplt.axes()['x'].label = 'Strike'
qplt.axes()['y'].label = 'Implied vol'

qplt.show()

A Jupyter Widget

A Jupyter Widget

<h3 style="color:#001f3f; font-family:Times New Roman">Generate fake/actual market data - TO TEST ON THE SKEWED MODEL (see smiles SPX/VIX)</h3>

In [13]:
spx_spot = 2901.61

# create these
spx_mkt_expiries = np.array([1/12, 2/12, 3/12, 4/12, 5/12, 6/12])
#Array with SPX maturities

spx_mkt_strikes_by_expiry = np.vstack([np.linspace(2840, 2940, 5) for x in range(6)])
#Array of arrays with strikes

vix_spot = 14.22

vix_mkt_expiries = np.array([1/12, 2/12, 3/12, 4/12, 5/12, 6/12])
#Array with VIX maturities

vix_mkt_strikes_by_expiry = np.vstack([np.array([11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 16, 17, 18,
                                                 19, 20, 21, 22, 23, 24, 25, 26]) for x in range(6)])
#Array of arrays
def getXiMkt(time):
    mats = [0, 1/12, 1/6, 1/4, 1/3, 5/12, 1/2, 7/12, 2/3, 3/4]
    xis = [14.22, 14.875, 15.225, 15.425, 15.925, 16.175, 16.475, 16.625, 16.875, 17.05]
    interp = interpolate.interp1d(mats, xis, kind='zero', fill_value='extrapolate')
    return interp(time)

xio_mkt_curve = np.array([getXiMkt(t) for t in times])
#Array with the forward variances implied from the market of variance swaps
#Piecewise cst

def f_spx(time):
    mats = [0, 1/4, 1/2, 3/4, 1]
    log_futs = np.log([2901.61, 2907.75, 2916.25, 2925.75, 2936.50])
    interp = interpolate.interp1d(mats, log_futs, fill_value='extrapolate')
    return np.exp(interp(time))

def local_vol_benchmark(maturity, strike):
    #done using Julien's parameters
    return sigma_loc


def instantaneous_rate_from_DF(maturity):
    mats = [0, 1/12, 1/4, 1/2]
    rs = [0.02158058, 0.02241764, 0.0259709 , 0.02786544]
    interp = interpolate.interp1d(mats, rs, kind='zero', fill_value='extrapolate')
    return interp(maturity)

def repo_rate_from_forward(maturity):
    rmats = [0, 1/12, 1/4, 1/2]
    rs = [0.02158058, 0.02241764, 0.0259709 , 0.02786544]
    interp = interpolate.interp1d(rmats, rs, kind='zero', fill_value='extrapolate')
    
    qmats = [0, 1/4, 1/2, 3/4, 1]
    discounts = np.array([integrate.quad(interp, 0, x)[0] for x in qmats])
    log_futs = np.log([2901.61, 2907.75, 2916.25, 2925.75, 2936.50])
    qints = log_futs-np.log(2901.61)+discounts
    qs = [(qints[i+1]-qints[i])/(qmats[i+1]-qmats[i]) for i in range(len(qmats)-1)]
    interp2 = interpolate.interp1d(qmats[:-1], qs, kind='zero', fill_value='extrapolate')
    
    return interp2(maturity)

<h1 style="color:#ec7f37; font-family:Times New Roman">PART 2: Transform the model into a SLV model and learn the $VIX^2$ function</h1>
<h3 style="color:#001f3f; font-family:Times New Roman">Kernel density estimation functions</h3>

In [15]:
# define a few kernels
kernel_quartic = lambda x : (15/16) * (1-x**2)**2 * (np.abs(x)<1.0)
kernel_gaussian = lambda x : (1/np.sqrt(2*np.pi)) * np.exp(-x**2/2)

In [17]:
# conditional expectation via Nadaraya-Watson (locally weighted average)
def conditional_expectation(x, xdata, ydata, sug_bandwidth, kern=kernel_gaussian):
    
    bandwidth = sug_bandwidth
    weights = kern((xdata[:, np.newaxis] - x) / bandwidth)
    while (np.sum(weights, axis=0) < 1e-14).any():
        bandwidth = 2 * bandwidth
        weights = kern((xdata[:, np.newaxis] - x) / bandwidth)
        print('Increasing bandwidth to ', bandwidth)
    
    return np.sum(weights * ydata[:, np.newaxis], axis=0) / np.sum(weights, axis=0)

In [18]:
# test: generate noisy data from a known function
X = np.random.randn(100)
g = lambda x : x*(1+x)/(1+x**2)
x = np.linspace(X.min(), X.max(), 101)
Y = g(X) + 0.25 * np.random.randn(100)
bw_silverman = (4/(3*len(X)))**0.2*np.std(X)

# scatter plot
nw_cond_exp_fig = qplt.figure(title='Nadaraya-Watson kernel density estimation',
           scales={"x": qplt.LinearScale()},
           min_width=800, min_height=300, width=200, height=200,
           padding_x=.02, padding_y=0.01,
           align='right',
           legend_location='bottom-right')

qplt.plot(x, conditional_expectation(x, X, Y, bw_silverman, kernel_gaussian), colors=['#FF5A09'],\
          display_legend=True, labels=['NW kernel reg (bw={:.2f})'.format(bw_silverman)],\
          marker='cross', marker_size=16)
qplt.scatter(X, Y, colors=['green'])
qplt.plot(x, g(x), colors=['#39CCCC'], display_legend=True, labels=['True Function'])

qplt.show()

A Jupyter Widget

<h3 style="color:#001f3f; font-family:Times New Roman">Simulate the SLV skewed Bergomi 2 factor model - TO FIX!</h3>

In [19]:
# method to interpolate the expectation conditional on S_t = x given the grid (X,Y)
def interpolate_x_y(x, X, Y, flat=False, bound_array=None):
    
    # interpolation
    interp_result = np.interp(np.atleast_1d(x), X, Y)
    assert not np.isnan(interp_result).any(), 'interpolation problem'
    
    # extrapolation
    below, above = x < X[0], x > X[-1]
    ## flat
    if flat:
        interp_result[below], interp_result[above] = Y[0], Y[-1]
    ## linear
    else:
        interp_result[below], interp_result[above] = Y[0]+ (x[below]-X[0])/(X[1]-X[0])*(Y[1]-Y[0]),\
                                 Y[-1] + (x[above]-X[-1])/(X[-2]-X[-1])*(Y[-2]-Y[-1])  
    # if bounds, cut the array
    if not (bound_array is None):
        return np.clip(interp_result, bound_array[0], bound_array[1])
    
    return interp_result

In [24]:
# Local variance computed over a spot grid
def model_local_var(stock_path, xi_path, xi0=xi0, times=times, S0=S0):

    # grid parameters
    grid_size = 60
    kappa_bandwidth = 1.5
    w_grid_spot = 0.8
    t_min = 0.25
    
    # allocations / initializations
    nb_steps = len(times)
    nb_paths = stock_path.shape[0]
    local_variance = np.zeros((nb_steps, grid_size))
    grid_spot = np.zeros((nb_steps,grid_size))
    
    # at time 0
    local_variance[0] = (xi0(0.0001)**2) * np.ones(grid_size)
    stock_j = stock_path[:,0]
    grid_spot[0] = np.linspace(w_grid_spot*np.min(stock_j)+(1.0-w_grid_spot)*S0,\
                               w_grid_spot*np.max(stock_j)+(1.0-w_grid_spot)*S0, grid_size)

    for j in range(1,nb_steps):
        dt = times[j] - times[j-1]
        tj = j * dt
        
        stock_j = stock_path[:,j]
        xi_j = xi_path[:,j]
        
        # spot grid at time j
        grid_spot[j] = np.linspace(w_grid_spot*np.min(stock_j)+(1.0-w_grid_spot)*S0,\
                                   w_grid_spot*np.max(stock_j)+(1.0-w_grid_spot)*S0, grid_size)
        
        # compute conditional expectation # see julien's paper for the bandwidth justification
        bandwidth_j = kappa_bandwidth * S0 * xi0(tj) * np.sqrt(max(tj,t_min)) * pow(nb_paths,-0.2)
        local_variance[j] = conditional_expectation(grid_spot[j], stock_j, xi_j, bandwidth_j)

    return grid_spot, local_variance

In [25]:
# build the local_var, local_spot grid on which we interpolate in the simulation
stock_grid, local_var_grid = model_local_var(stock, xi)

Increasing bandwidth to  0.6000000000000001
Increasing bandwidth to  0.6000000000000001
Increasing bandwidth to  0.6589164243448168
Increasing bandwidth to  1.3178328486896336
Increasing bandwidth to  0.6643847042007187
Increasing bandwidth to  0.6698083427969557
Increasing bandwidth to  1.3396166855939113
Increasing bandwidth to  0.6751884159143444
Increasing bandwidth to  0.7167767045747945
Increasing bandwidth to  0.7218067782486047
Increasing bandwidth to  1.4436135564972095
Increasing bandwidth to  0.7268020404691687
Increasing bandwidth to  1.4536040809383375
Increasing bandwidth to  2.907208161876675
Increasing bandwidth to  0.7317632041409815
Increasing bandwidth to  1.463526408281963
Increasing bandwidth to  2.927052816563926
Increasing bandwidth to  0.7366909581630539
Increasing bandwidth to  1.4733819163261077
Increasing bandwidth to  2.9467638326522154
Increasing bandwidth to  0.7415859685455995
Increasing bandwidth to  1.483171937091199
Increasing bandwidth to  0.746448879

In [53]:
# stochastic local skewed bergomi 2 factor
def xi_stock_skewed_slv(stock_grid, local_var_grid, local_vol_function,\
                 times, increments,\
                 k1, k2, nu, rho12,\
                 theta, xi0, gamma, beta, zeta, S0, r, q):

    # simulation variables
    nb_paths, nb_steps = increments.shape[0], increments.shape[1]
    a = 1 / np.sqrt((1-theta)**2 + theta**2 + 2*rho12*theta*(1-theta))
    
    # time dependent parameters
    gamma_t, beta_t, zeta_t = create_array_time_dependent_parameters(times,\
                                                    vix_maturities, gamma, beta, zeta)
    nu_t = np.array([nu * zeta_t[i] / ((1-gamma_t[i]) + beta_t[i] * gamma_t[i])\
                     for i in range(nb_steps)])
    omega_t = 2 * nu_t
    
    # t = 0
    k = np.array([[k1], [k2]])
    norm = a*np.array([[1-theta], [theta]])
    chi_t_vector = a**2 * ((1-theta)**2 * (1 - np.exp(-2*k1*times))/(2*k1)
                        + theta**2 * (1 - np.exp(-2*k2*times))/(2*k2)
                        + 2*rho12*theta*(1-theta) * (1-np.exp(-(k1+k2)*times))/(k1+k2)
                        )
    
    # processes
    xi_t = np.ones((nb_steps, nb_paths)) * xi0(np.array([times]).T)
    local_vol = np.zeros((nb_steps-1,nb_paths))
    
    # stochastic local variables arrays
    conditional_expectation = np.zeros((nb_steps-1,nb_paths))
    leverage_function = np.zeros((nb_steps-1,nb_paths))
    vol_stochastic_local = np.zeros((nb_steps-1,nb_paths))
    stock_stochastic_local = np.ones((nb_steps,nb_paths)) * S0
    
    times_sim = times[1:]
    for ti, t in enumerate(times_sim):
        t0 = times[ti]
        dt = t - t0

        # instantaneous variance # beware indices
        little_xt = (norm*[increments[:,ti+1,0], increments[:,ti+1,1]]).sum(axis=0)
        xi_t[ti+1] = xi0(t0) *\
                            ((1-gamma_t[ti+1])*np.exp(omega_t[ti+1]*little_xt\
                             - 0.5*(omega_t[ti+1]**2)*chi_t_vector[ti+1]) + \
                            gamma_t[ti+1]*np.exp(beta_t[ti+1]*omega_t[ti+1]*little_xt\
                             - 0.5*((omega_t[ti+1]*beta_t[ti+1])**2)*chi_t_vector[ti+1]))

        # leverage function computation
        grid_stock_ti = stock_grid[ti]
        local_var_ti = local_var_grid[ti]
        stock_stoch_loc_ti = stock_stochastic_local[ti]
        local_vol[ti] = local_vol_function(T=max(t0,0.0001), K=stock_stoch_loc_ti)
        conditional_expectation[ti] = interpolate_x_y(stock_stoch_loc_ti,\
                                            grid_stock_ti, local_var_ti, flat=True)
        
        leverage_function[ti] = local_vol[ti] / np.sqrt(conditional_expectation[ti])

        # stochastic local vol
        vol_stochastic_local[ti] = leverage_function[ti] * np.sqrt(xi_t[ti])
        print(vol_stochastic_local[ti], local_vol[ti])

        # spot via log euler scheme
        stock_stochastic_local[ti+1] = stock_stochastic_local[ti] *\
                                np.exp(vol_stochastic_local[ti] * increments[:,ti+1,2]\
                                + (r(t0)-q(t0) - 0.5*dt*vol_stochastic_local[ti]**2))


    return stock_stochastic_local, vol_stochastic_local,\
            xi_t, local_vol, conditional_expectation, leverage_function

In [54]:
# benchmark ...
local_vol_function = lambda T,K: ((0.1 - 0.15 * np.log(K/S0))/np.sqrt(T))/10.0 + 0.1
# simulation
stock_stochastic_local, vol_stochastic_local,\
xi_t, local_vol,\
conditional_expectation,\
leverage_function = xi_stock_skewed_slv(stock_grid, local_var_grid, local_vol_function,\
                                        times, increments,\
                                        k1, k2, nu, rho12,\
                                        theta, xi0, gamma_beta_zeta[:,0],\
                                        gamma_beta_zeta[:,1],\
                                        gamma_beta_zeta[:,2], S0, r, q)

[5.5 5.5 5.5 ... 5.5 5.5 5.5] [1.1 1.1 1.1 ... 1.1 1.1 1.1]
[0.0184232  0.1054289  0.40963246 ... 0.28626732 0.32173962 0.32837661] [0.02136055 0.11156572 0.39612334 ... 0.30110068 0.30346058 0.30447351]
[0.03833463 0.0980706  0.29228544 ... 0.20281054 0.24317246 0.25525246] [0.04465026 0.10808674 0.30496328 ... 0.24292593 0.24698338 0.24845101]
[0.04808405 0.09538456 0.27179529 ... 0.17916248 0.24320694 0.23448982] [0.05512364 0.10656545 0.27041351 ... 0.21876025 0.21923766 0.21921689]
[0.05157355 0.10107682 0.22232402 ... 0.14499014 0.2196906  0.2209762 ] [0.06076412 0.10629887 0.24828363 ... 0.20100831 0.20427468 0.20159491]
[0.05014996 0.08741943 0.19218502 ... 0.12486077 0.18477599 0.19909135] [0.06504861 0.1055617  0.23161538 ... 0.19082481 0.19440107 0.19023939]
[0.05106815 0.08104211 0.17951355 ... 0.13840674 0.19076055 0.20047914] [0.06854383 0.10532346 0.22069954 ... 0.184103   0.18623286 0.1810724 ]
[0.05209623 0.0808173  0.17238093 ... 0.12649362 0.17834034 0.18903337] [0.0

[0.04488512 0.06159789 0.08873682 ... 0.10193357 0.07092132 0.16547486] [0.08966751 0.10207118 0.13978759 ... 0.12586642 0.12634247 0.12342038]
[0.03421061 0.04984459 0.07782454 ... 0.10417374 0.07031011 0.15919581] [0.08985564 0.10196621 0.13955859 ... 0.12577522 0.12591502 0.12247678]
[0.03486909 0.06102514 0.06907535 ... 0.10162712 0.0724352  0.1395338 ] [0.09000296 0.10192906 0.1392175  ... 0.12562631 0.12585701 0.1220986 ]
[0.03613826 0.06476548 0.06907426 ... 0.09907253 0.07283206 0.14844586] [0.09012231 0.10193269 0.13877454 ... 0.12548597 0.12561644 0.12137165]
[0.0428264  0.07106828 0.06309862 ... 0.10773017 0.0722462  0.1365615 ] [0.09009169 0.10207716 0.13850353 ... 0.12527628 0.12537468 0.1210226 ]
[0.04237384 0.0706187  0.06030647 ... 0.11294411 0.06800769 0.13601954] [0.09016817 0.10216238 0.13812037 ... 0.12455705 0.12554207 0.12050105]
[0.04081892 0.06886468 0.06541273 ... 0.10788421 0.06416487 0.14345592] [0.09023821 0.10218434 0.13793592 ... 0.12443605 0.12535904 0.12

[0.01479857 0.05287697 0.06278074 ... 0.09196054 0.08444383 0.14159875] [0.09330251 0.10158101 0.12863687 ... 0.11686195 0.11691092 0.11539876]
[0.05193376 0.05149313 0.0637494  ... 0.08811471 0.08607018 0.14437232] [0.09328355 0.10156624 0.12857137 ... 0.11650919 0.1169097  0.11530821]
[0.05039226 0.05259853 0.05667151 ... 0.08957973 0.08327796 0.14354963] [0.09324982 0.10151753 0.12841775 ... 0.11641854 0.11671465 0.11500168]
[0.01416103 0.05537052 0.07117127 ... 0.09358531 0.084202   0.13917524] [0.09329862 0.10151268 0.12834859 ... 0.11642341 0.11645614 0.11471653]
[0.0145526  0.06175745 0.06218206 ... 0.0938953  0.08330493 0.14352416] [0.09332916 0.1015033  0.12812524 ... 0.11632843 0.1163289  0.11441756]
[0.01190092 0.05713989 0.05803289 ... 0.09246365 0.08431901 0.14396265] [0.09337267 0.1014346  0.12815479 ... 0.11632904 0.11616688 0.11444466]
[0.01293024 0.05825627 0.05337238 ... 0.09174428 0.08358848 0.13679664] [0.09340403 0.1014903  0.12797891 ... 0.11634562 0.11603227 0.11

[0.0285282  0.06659192 0.06276855 ... 0.11151023 0.08020365 0.30268282] [0.09495832 0.10042195 0.12348781 ... 0.11387832 0.11282075 0.11285763]
[0.05216939 0.0673306  0.07682449 ... 0.11538814 0.07896189 0.27978621] [0.09498928 0.10055596 0.12349488 ... 0.11395763 0.11266817 0.11286318]
[0.0670466  0.06768569 0.06408374 ... 0.11732487 0.07616281 0.223789  ] [0.09500281 0.1004997  0.12349231 ... 0.11389979 0.11257135 0.11247038]
[0.0537765  0.06126565 0.07141561 ... 0.11042634 0.07800421 0.22379728] [0.09505689 0.10048814 0.12347603 ... 0.11402629 0.11254516 0.11267476]
[0.07167174 0.06681658 0.07723331 ... 0.09988761 0.07821324 0.21522443] [0.09510182 0.10047414 0.12343767 ... 0.11408613 0.11248    0.11221413]
[0.04255357 0.05875837 0.08308792 ... 0.09329688 0.07698382 0.23287372] [0.09498876 0.10056748 0.12333869 ... 0.11405338 0.11265733 0.11225427]
[0.04111844 0.05913839 0.08094149 ... 0.09668793 0.07842397 0.25087061] [0.09504125 0.10062079 0.12334888 ... 0.11372694 0.11280663 0.11

<h3 style="color:#001f3f; font-family:Times New Roman">Test the slv version of the model on a local vol benchmark - TO PASS</h3>

In [52]:
# SPX call prices
price_spx_optionsq = np.ravel(european_call_spx(stock_stochastic_local, times, maturity, range_strike_spx, r))

price_spx_optionsq_fig = qplt.figure(title='SPX call prices - stochastic local skewed 2 factor Bergomi', 
           scales={"x": qplt.LinearScale()},
           min_width=800, min_height=300, width=1000, height=400,
           padding_x=.02, padding_y=0.01,
           align='right',
           legend_location='right')
price_spx_optionsq_fig.layout.width = '700px'

qplt.plot(np.array(range_strike_spx), price_spx_optionsq,\
          colors=['#FF851B'], marker='circle', marker_size=16)
qplt.axes()['x'].label = 'Strike'
qplt.axes()['y'].label = 'Price'

qplt.show()

# SPX call implied volatilities
slv_moneyness = lfmoneyness(S0, range_strike_spx, r(maturity), maturity)
iv_spx_optionsq = impvol_bisection(slv_moneyness, maturity, price_spx_optionsq/S0, True)

iv_spx_options_figq = qplt.figure(title='SPX call implied volatilities - stochastic local skewed 2 factor Bergomi', 
           scales={"x": qplt.LinearScale()},
           min_width=800, min_height=300, width=1000, height=400,
           padding_x=.02, padding_y=0.01,
           align='right',
           legend_location='right')
iv_spx_options_figq.layout.width = '700px'

qplt.plot(np.array(range_strike_spx), iv_spx_optionsq, colors=['#0074D9'], marker='cross', marker_size=16,\
          display_legend=True, labels=['SLV'])
qplt.plot(np.array(range_strike_spx), local_vol_function(maturity,np.array(range_strike_spx)),\
          colors=['#7FDBFF'], marker='cross', marker_size=16, display_legend=True, labels=['benchmark'])
qplt.axes()['x'].label = 'Strike'
qplt.axes()['y'].label = 'Implied vol'

qplt.show()

A Jupyter Widget

A Jupyter Widget

<h3 style="color:#001f3f; font-family:Times New Roman">$VIX^2$ learning process - TODO</h3>